# Wisselstromen

Dit script is bedoeld om het (voorlopige) bekostigingsbestand van DUO te anonimiseren en om te zetten in een makkelijk leesbaar bestandformaat. 

Het door DUO aangeleverde bestand bevat verschillende recordsoorten met verschillende lengte in één bestand, zonder bijbehorende headers. 

Na het runnen van dit script worden verschillende bestanden aangemaakt in de output folder:
1. CSV bestand voor recordsoort BLB: Bekostigingsloopbaan
2. CSV bestand voor recordsoort BRD: Bekostigingsresultaat deelname
3. CSV bestand voor recordsoort BRR: Bekostigingsresultaat resultaat
4. outputbestand
5. sleutelbestand

Bestanden 1 t/m 3 zijn bedoeld voor analyse van het bekostingsbestand; deze bevatten dezelfde gegevens als het bekostigingsbestand, gesplitst naar recordsoort.

Bestand 4 is de geanonimiseerde versie van het originele bestand, bedoeld om het R-script te testen.

Bestand 5 bevat alle persoonsnummers en bijbehorend id om later te kunnen koppelen met attibuten uit eigen databases.



## Definities

Zie **PvE HO-instelling - DUO, Bijlage 8: Bestandsbeschrijving analysebestand voorlopige en definitieve bekostiging voor HO-instellingen, p.168 e.v.**

### 17.5 Formaat
Het bestand bevat de volgende recordsoorten:
VLP = Voorlooprecord  
BLB = Bekostigingsloopbaan student  
BRD = Bekostigingsresultaat deelname  
BRR = Bekostigingsresultaat resultaat  
SLR = Sluitrecord

- Een voorlooprecord (**VLP**);
- Setjes van alle records met de bekostiging van de betrokken persoon. De persoon wordt opgenomen indien hij minimaal één bekostigingsresultaat voor een
deelname of resultaat heeft bij de betrokken BRIN.
    - Nul of één **BLB**-record:  
    - Nul, één of meer **BRD**-records; Dit zijn alle bekostigingsresultaten van in het bekostigingsjaar beoordeelde deelnames. Daarnaast worden er
deelnames geselecteerd, die buiten de beoordeling vallen. Deze krijgen de status MV, om zo alle inschrijvingen van de betreffende persoon, die geldig
zijn in de periode 2 oktober van het bekostigingsjaar - 3 tot en met 1 oktober van het bekostigingsjaar - 2 worden, in het bestand op te nemen.
Dit kunnen ook bekostigingsresultaten bij andere instellingen zijn.  
    - Nul, één of meer **BRR**-records; Dit zijn alle bekostigingsresultaten van in het bekostigingsjaar beoordeelde resultaten. Dit kunnen ook
bekostigingsresultaten bij andere instellingen zijn. 
    - Voor een persoon zal er altijd op zijn minst één BRD- of één BRR-recordzijn;  
    - De records die betrekking hebben op dezelfde persoon, hebben alle hetzelfde burgerservice- en/of onderwijsnummer.  
- Een sluitrecord met aantal BLB, BRD en BRR-records (**SLR**);

De velden van een record worden gescheiden door een pipe-teken (|). Het eerste veld
van een record geeft altijd de recordsoort aan.

De BLB-records zijn oplopend gesorteerd op burgerservicenummer. Als er geen
burgerservicenummer is dan oplopend op onderwijsnummer, waarbij records met alleen
een onderwijsnummer achteraan staan.

### 17.6 Datumformaat
Datumvelden zijn in het formaat JJJJMMDD; Het betreft dus een numeriek veld van 8 positite slang. In de onderstaande beschrijving wordt dit in de kolom 'Formaat' aangegeven als 'Datum 8 (jjjjmmdd)'


## Bestanden laden

In [11]:
import pandas as pd
import os


### Headers voorbereiden
In de bestanden staan geen headers. Deze worden hieronder apart ingeladen.

In [12]:
# Dictionary with column names for each recordsoort (source = PvE)
columns_to_sum = {
    'VLP': [
        'Recordsoort',
        'BRIN',
        'Bekostigingsjaar',
        'Datum aanmaak'
    ],
    'BLB': [
        'Recordsoort',
        'Burgerservicenummer',
        'Onderwijsnummer', 
        'DatumGraadBehaaldAD',
        'DatumGraadBehaaldADLG', 
        'DatumGraadBehaaldBa', 
        'DatumGraadBehaaldBaLG', 
        'DatumGraadBehaaldMa', 
        'DatumGraadBehaaldMaLG', 
        'VerbruikAD', 
        'VerbruikADLG', 
        'VerbruikBA', 
        'VerbruikBALG', 
        'VerbruikMA', 
        'VerbruikMALG', 
        'AantalBekostigdeInschrijvingenBa', 
        'AantalBekostigdeInschrijvingenBaLG', 
        'AantalBekostigdeInschrijvingenMa', 
        'AantalBekostigdeInschrijvingenMaLG', 
        'AantalBekostigdeInschrijvingenBaLGnaGraadBaMa', 
        'AantalBekostigdeInschrijvingenMaLGnaGraadMa'
    ],
    'BRD': [
        'Recordsoort',
        'Burgerservicenummer',
        'Onderwijsnummer',
        'Brin',
        'inschrijvingvolgnummer',
        'Bekostigingsindicatie',
        'CodeBekostigingstatus',
        'bekostigingsniveau',
        'opleidingscode',
        'opleidingsniveau',
        'opleidingsfase',
        'datumInschrijving',
        'datumUitschrijving',
        'eersteInschrijving',
        'inschrijvingsvorm',
        'onderwijsvorm',
        'datumEersteAanlevering',
        'bekostigingsduur',
        'OnderopleidingOnderdeel',
        'bekostigingscode',
        'indicatieSectorLG',
        'indicatieBaMa',
        'indicatieAcademischZiekenhuisenhuis',
        'indicatieNationaliteitsvoorwaardeSF',
        'indicatieGBARelatie'
    ],
    'BRR': [
        'Recordsoort',
        'Burgerservicenummer',
        'Onderwijsnummer',
        'Brin',
        'resultaatvolgnummer',
        'Bekostigingsindicatie',
        'CodeBekostigingstatus',
        'bekostigingsniveau',
        'JointDegreeFactor',
        'opleidingscode',
        'opleidingsniveau',
        'opleidingsfase',
        'eersteGraad',
        'datumDiploma',
        'onderwijsvorm',
        'datumEersteAanlevering',
        'opleidingOnderdeel',
        'bekostigingscode',
        'indicatieSectorLG',
        'indicatieBaMa',
        'indicatieAcademischZiekenhuis',
        'indicatieGraadTeltVoorBekostigingsloopbaan',
        'indicatieNationaliteitsvoorwaardeSF',
        'indicatieGBARelatie'
    ],
    'SLR': [
        'Recordsoort',
        'AantalBLBrecords',
        'AantalBRDrecords',
        'AantalBRRrecords'
    ]
}

# Directory to save CSV files
output_folder = 'headers'
os.makedirs(output_folder, exist_ok=True)

# Generate dataframe for each key in the dictionary
for recordsoort, columns in columns_to_sum.items():
    df = pd.DataFrame(columns=columns)
    
    # Save the dataframe to a CSV file named <key>.csv in the 'headers' folder
    df.to_csv(f'{output_folder}/{recordsoort}.csv', index=False)

print("CSV files have been created in the 'headers' folder.")


CSV files have been created in the 'headers' folder.


### Functie om csv bestanden op te delen in verschillende dataframes
De bekostigingsbestanden bevatten verschillende record soorten, met elk een verschillend aantal kolommen. Deze functie maakt per record soort een apart dataframe aan met als naam DF_[record soort].

In [13]:
def load_and_group_data(folder, file, sep='|'):
    """
    Loads a CSV file, groups rows by the first column (recordsoort),
    and stores each group as a DataFrame in a dictionary.
    
    Args:
        folder (str): The folder containing the file.
        file (str): The name of the file.
        sep (str): The separator used in the CSV file (default is '|').
        
    Returns:
        dict: A dictionary where keys are the recordsoort from the first column, 
              and values are DataFrames containing the rows for each recordsoort.
    """
    path = os.path.join(folder, file)

    # Step 1: Create a dictionary with the header names per recordsoort
    input_folder = 'headers' # Directory containing the CSV files with the headers

    columns_to_sum = {}

    for filename in os.listdir(input_folder):
        if filename.endswith('.csv'):
            file_path = os.path.join(input_folder, filename)
            
            df = pd.read_csv(file_path)

            recordsoort = filename.replace('.csv', '') # Get the recordsoort from the file name (without the .csv extension)
            columns_to_sum[recordsoort] = df.columns.tolist()

    # Step 2: Open file bekostigingsbestand and make a list of all the unique recordsoorten
    recordsoorten = set()

    with open(path, 'r') as f:
        for line in f:
            row = line.strip().split(sep)
            recordsoorten.add(row[0])  # Add the first column value (recordsoort) to the set

    # Step 3: Initialize a dictionary with the recordsoort as key
    df_dict = {recordsoort: [] for recordsoort in recordsoorten}

    # Step 4: Open the file again and append the row to the appropriate dataframe list
    with open(path, 'r') as f:
        for line in f:
            row = line.strip().split(sep)
            recordsoort = row[0]
            df_dict[recordsoort].append(row)  

    # Step 5: Convert lists to dataframes
    for recordsoort, rows in df_dict.items():
        df_dict[recordsoort] = pd.DataFrame(rows)

    # Step 6: Assign columns from step 1 to each dataframe
    for recordsoort, df in df_dict.items():
        if recordsoort in columns_to_sum:
            columns = columns_to_sum[recordsoort]
            max_columns = len(columns)
            
            # Adjust the DataFrame to match the expected number of columns
            df = df.iloc[:, :max_columns]  # Limit to expected number of columns
            df.columns = columns
            
            # Add missing columns if any specified columns are absent
            for col in columns:
                if col not in df.columns:
                    df[col] = ''  # Initialize as empty string if missing
        
        # Update the dataframe in the dictionary
        df_dict[recordsoort] = df

    return df_dict


### Bestand 1: DEFBEK_LO_2025_20240522_25DW.csv

In [14]:
folder = 'data'
file = 'DEFBEK_LO_2025_20240522_25DW.csv'

df_dict = load_and_group_data(folder, file)


### Bestand 2: DEFBEK_2025_20240522_25DW.csv

In [15]:
folder = 'data'
file = 'DEFBEK_2025_20240522_25DW.csv'

df_dict = load_and_group_data(folder, file)


### Dataframes maken

In [16]:
df_vlp = df_dict.get('VLP')
df_brr = df_dict.get('BRR')
df_blb = df_dict.get('BLB')
df_brd = df_dict.get('BRD')
df_slr = df_dict.get('SLR')


### Check aantal records per datafame

In [17]:
# Extract the expected counts from df_slr
expected_brr = df_slr['AantalBRRrecords'].iloc[0]
expected_blb = df_slr['AantalBLBrecords'].iloc[0]
expected_brd = df_slr['AantalBRDrecords'].iloc[0]

# Get the actual counts from the DataFrames
actual_brr = len(df_brr)
actual_blb = len(df_blb)
actual_brd = len(df_brd)

# Compare and print the results
print(f"BRR records: expected {expected_brr}, actual {actual_brr}")
print(f"BLB records: expected {expected_blb}, actual {actual_blb}")
print(f"BRD records: expected {expected_brd}, actual {actual_brd}")

BRR records: expected 12794, actual 12794
BLB records: expected 53336, actual 53336
BRD records: expected 102462, actual 102462


## Data anonimiseren

### Sleuteltabel
Met deze tabel kunnen de geanonimiseerde data later weer gekoppeld worden.

In [18]:
# Extract the key columns from each DataFrame
df_brr_key = df_brr[['Burgerservicenummer', 'Onderwijsnummer']]
df_blb_key = df_blb[['Burgerservicenummer', 'Onderwijsnummer']]
df_brd_key = df_brd[['Burgerservicenummer', 'Onderwijsnummer']]

# Concatenate the keys into a single DataFrame
df_key = pd.concat([df_brr_key, df_blb_key, df_brd_key])

# Remove duplicates to ensure unique combinations of Burgerservicenummer and Onderwijsnummer
df_key = df_key.drop_duplicates(subset=['Burgerservicenummer', 'Onderwijsnummer'])

# Sort by Onderwijsnummer and Burgerservicenummer
df_key = df_key.sort_values(
    by=['Onderwijsnummer', 'Burgerservicenummer'],
    ascending=[True, True],
    na_position='last'
)

# Assign sequential student_id starting from 1
df_key['student_id'] = range(1, len(df_key) + 1)

# Convert to string to be able to merge
df_key['student_id'] = df_key['student_id'].astype(str)


### Schoon dataframes en sla op als csv bestanden
Gebruik de sleuteltabel om de het `Student_id` te koppelen met alle bestanden o.b.v. `Burgerservicenummer` en `Onderwijsnummer`.

In [19]:
output_folder = 'output'
os.makedirs(output_folder, exist_ok=True)

# Merge student_id into df_brr and save to .csv
df_brr = pd.merge(df_brr, df_key[['Burgerservicenummer', 'Onderwijsnummer', 'student_id']], on=['Burgerservicenummer', 'Onderwijsnummer'], how='left')
df_brr['Burgerservicenummer'] = df_brr['student_id'].astype(int)
df_brr.drop(columns=['student_id'], inplace=True)
df_brr['Onderwijsnummer'] = ''
brr_filename = os.path.join(output_folder, 'BRR.csv')
df_brr.to_csv(brr_filename, index=False)

# Merge student_id into df_blb and save to .csv
df_blb = pd.merge(df_blb, df_key[['Burgerservicenummer', 'Onderwijsnummer', 'student_id']], on=['Burgerservicenummer', 'Onderwijsnummer'], how='left')
df_blb['Burgerservicenummer'] = df_blb['student_id'].astype(int)
df_blb.drop(columns=['student_id'], inplace=True)
df_blb['Onderwijsnummer'] = ''
blb_filename = os.path.join(output_folder, 'BLB.csv')
df_blb.to_csv(blb_filename, index=False)

# Merge student_id into df_brd and save to .csv
df_brd = pd.merge(df_brd, df_key[['Burgerservicenummer', 'Onderwijsnummer', 'student_id']], on=['Burgerservicenummer', 'Onderwijsnummer'], how='left')
df_brd['Burgerservicenummer'] = df_brd['student_id'].astype(int)
df_brd.drop(columns=['student_id'], inplace=True)
df_brd['Onderwijsnummer'] = ''
brd_filename = os.path.join(output_folder, 'BRD.csv')
df_brd.to_csv(brd_filename, index=False)


### Schoon sleutelbestand en sla op als csv bestand

In [20]:
output_folder = 'output'
os.makedirs(output_folder, exist_ok=True)
key_filename = os.path.join(output_folder, 'sleutelbestand.csv')

# Remove duplicates
df_key = df_key.drop_duplicates(subset=['Burgerservicenummer', 'Onderwijsnummer'])

# Save df_key to a CSV file
df_key.to_csv(key_filename, index=False)


### Sla schoon bekostigingsbestand op

Hier creëren we het geanonimiseerde bekostigingsbestand in dezelfde structuur en volgorde als het origineel. 
> ! Het is nog niet gelukt om de volgorde exact gelijk te krijgen

In [21]:
# Step 1: Combine df_brr, df_blb, and df_brd into one list
combined_data = (
    df_brr.values.tolist() +
    df_blb.values.tolist() +
    df_brd.values.tolist()
)

# Step 2: Sort combined data by student_id
sorted_data = sorted(
    combined_data,
    key=lambda x: (
        x[1],  # Sort by student_id (x[1]) for all record types
        0 if x[0] == 'BLB' else 1 if x[0] == 'BRD' else 2,  # Sort by record type (BLB first, then BRD, BRR)
        (x[3], float(x[4]) if x[0] in ['BRD', 'BRR'] and x[4].replace('.', '', 1).isdigit() else float('inf'), x[4]) # For BRD and BRR records, sort by x[3] first and then by x[4]
    )
)

# Step 3: Add the first line from df_vlp and the last line from df_slr
first_line = df_vlp.iloc[0].values.tolist()  # First row from df_vlp
last_line = df_slr.iloc[-1].values.tolist()  # Last row from df_slr

# Step 4: Combine everything into one list
final_data = [first_line] + sorted_data + [last_line]

# Step 5: Define the output folder and file path
output_folder = 'output'
os.makedirs(output_folder, exist_ok=True)
output_filename = os.path.join(output_folder, 'output.csv')

# Step 6: Write to a .csv file with '|' as the delimiter and no headers
with open(output_filename, 'w') as f:
    for row in final_data:
        f.write('|'.join(map(str, row)) + '\n') # Write each row separately, keeping the number of delimiters equal to the number of columns in each row


## Vergelijk input en output file

Poging om het geanonimiseerde bestand te vergelijken, maar door de afwijkende volgorde kan de vergelijking niet (op deze manier) gemaakt worden.

In [22]:
# File paths
folder = 'data'
file = 'DEFBEK_2025_20240522_25DW.csv'
input_file = os.path.join(folder, file)

output_folder = 'output'
os.makedirs(output_folder, exist_ok=True)  # Ensure the output folder exists
output_filename = os.path.join(output_folder, 'output.csv')

# Open the input and output files
with open(input_file, 'r') as infile, open(output_filename, 'r') as outfile:
    # Read all lines from both files
    input_lines = infile.readlines()
    output_lines = outfile.readlines()

    # Ensure both files have the same number of lines
    if len(input_lines) != len(output_lines):
        print("Error: The files have different number of rows.")
    else:
        # Process each line and check conditions
        for input_line, output_line in zip(input_lines, output_lines):
            # Split the lines by delimiter ('|')
            input_columns = input_line.strip().split('|')
            output_columns = output_line.strip().split('|')

            # Ensure both lines have the same number of columns
            if len(input_columns) != len(output_columns):
                print("Error: The rows have different number of columns.")
                break
            else:
                # Compare columns, excluding 'Burgerservicenummer' (index 1) and 'Onderwijsnummer' (index 2)
                input_columns_to_check = input_columns[3:]  # Excluding the first three columns (index 1 and 2)
                output_columns_to_check = output_columns[3:]  # Excluding the first three columns (index 1 and 2)

                # Check if any other columns have changed
                if input_columns_to_check != output_columns_to_check:
                    print(f"Error: Data mismatch detected for line:\nInput: {input_line.strip()}\nOutput: {output_line.strip()}")
                    break
        else:
            # If no errors are found, print success
            print("Success: Files are similar as expected.")


Error: Data mismatch detected for line:
Input: BRD|013852139|112603082|25DW|4008006|N|mv|LAAG|35255|HBO-BA|B|20220901|20230831|J|S|DT|20220803|48|ONDERWIJS|BEKOSTIGD||||J|J
Output: BRD|31||25DW|3951084|N|mv|LAAG|35195|HBO-BA|D|20220901|20230831|J|S|DT|20220718|48|ONDERWIJS|BEKOSTIGD||||J|J


De mismatch wordt veroorzaakt door een verkeerde sortering bij het toekennen van student_id's bij het aanmnaken van de sleuteltabel. 

> heeft dit gevolgen voor het inlezen?